## Common Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import json, os, gzip
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering, MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, Normalizer
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.datasets import make_moons, make_circles, make_blobs

Constants

In [ ]:
FIGURE_SAVE_PATH: str = "./figures"
TABLE_SAVE_PATH: str = "./table_results"
DATASET_PATH: str = "./datasets"
FILES:list[str] = ["dblp-ref-0.json", "dblp-ref-1.json", "dblp-ref-2.json", "dblp-ref-3.json"]

# Sampling length our dataset is too big this makes it more manageable to work with
# 2000 per year is a good representation of our dataset
DEV_MODE:bool = False
SAMPLES_PER_YEAR:int = 250 if DEV_MODE else 2000

### Load the Dataset

In [ ]:
# loading data
def read_json_lines(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            yield json.loads(line)

def load_all(files=FILES):
    rows = []
    for fp in files:
        for rec in read_json_lines(f"./datasets/{fp}"):
            rows.append(rec)
    return pd.DataFrame(rows)

df = load_all()

as_type = {
    "year": "category",
    "n_citation": np.uint32,
}

df = df.astype(as_type)
df.info()

In [ ]:
df.head(10)

## Task 1: Preprocessing & Feature Generation

#### Preprocess & Sample The Dataset

In [ ]:
# Drop Missing years
df = df[(df["title"].notna()) & (df["year"].notna())]


# Filter out rare venues
venue_counts = df["venue"].value_counts()
df = df[df["venue"].isin(venue_counts[venue_counts >= 50].index)]

# Filter out missing venues
print(len(df[df['venue'].str.strip() == ""]))
df = df[df['venue'].str.strip() != ""]

# as we saw later when doing simple EDA the papers with n_citation = 50 are likely a data scraping error as it shift the distribution outside a power like distribution
# SEE:
# https://link.springer.com/article/10.1007/s100510050359
df_clean_source = df[df['n_citation'] != 50]


print(f"Sampling {SAMPLES_PER_YEAR} papers per year...")

sampled_df = (
    df_clean_source.groupby('year', observed = True)
    .apply(lambda x : x.sample(min(len(x), SAMPLES_PER_YEAR)), include_groups = False)
    .reset_index(level = 'year')
)

# Handle normalizing or fixing the dataset on the sample since it will be less
# computationally expensive

# Replace missing abstracts with empty string
sampled_df["abstract"] = sampled_df["abstract"].fillna("")

# Normalize Text
sampled_df["title"] = sampled_df["title"].str.lower()
sampled_df["abstract"] = sampled_df["abstract"].str.lower()

# Combine title and abstract as one feature
sampled_df['text'] = sampled_df['title'] + " " + sampled_df['abstract']

# Reset Index so df rows align with matrix rows (0 to N)
sampled_df = sampled_df.reset_index(drop = True)

print(f"Original Shape: {df.shape}")
print(f"Sampled Shape: {sampled_df.shape}")
sampled_df.head()


### Generate Black Box Features
These are for our clustering and classification tasks in our proposal.

#### TF-IDF Pipeline

In [ ]:
# Run TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# In task 2 I Found more stop words so we add this here in order to get better topic clustering

my_stop_words = list(TfidfVectorizer(stop_words="english").get_stop_words())

tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words=my_stop_words,
    ngram_range=(1,2)
)
X_tfidf = tfidf.fit_transform(sampled_df["text"])

Important: You only really need to run this once after that its meaningless

We test here to find the optimal n_dimension by trying to maximize the silhoutte score for sparse text datasets its usually advised for the n_dimension to be around 50-350


In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pandas as pd
import matplotlib.pyplot as plt


components_to_test = [50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300]
n_clusters = 10
max_sil_samples = 5000   # subsample for silhouette to keep it fast

results = []

for n_comp in components_to_test:
    print(f"\n=== Testing {n_comp} components ===")

    svd = TruncatedSVD(n_components=n_comp, random_state=42)
    X_red = svd.fit_transform(X_tfidf)

    cum_var = svd.explained_variance_ratio_.sum()
    print(f"Cumulative explained variance: {cum_var:.4f}")

    # K-means on reduced data
    km = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    labels = km.fit_predict(X_red)

    # Subsample for silhouette (it’s O(n^2))
    if X_red.shape[0] > max_sil_samples:
        idx = np.random.choice(X_red.shape[0], max_sil_samples, replace=False)
        sil = silhouette_score(X_red[idx], labels[idx])
    else:
        sil = silhouette_score(X_red, labels)

    print(f"Silhouette score: {sil:.4f}")

    results.append({
        "n_components": n_comp,
        "cum_explained_var": cum_var,
        "silhouette": sil
    })

results_df = pd.DataFrame(results)
display(results_df)

# plot the tradeoff
fig, ax1 = plt.subplots(figsize=(9,5))

ax1.plot(results_df["n_components"], results_df["silhouette"],
         marker="o", label="Silhouette score")
ax1.set_xlabel("Number of components")
ax1.set_ylabel("Silhouette score")
ax1.grid(True, linestyle="--", alpha=0.5)

ax2 = ax1.twinx()
ax2.plot(results_df["n_components"], results_df["cum_explained_var"],
         marker="s", color="tab:orange", label="Cumulative variance")
ax2.set_ylabel("Cumulative explained variance")

# Combine legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="best")

plt.title("Tradeoff: PCA components vs clustering & variance")
plt.tight_layout()
plt.savefig(f"{FIGURE_SAVE_PATH}/pca_ncomponent_tradeoff.png")
plt.show()


#### PCA Preprocess Pipeline

In [ ]:
# PCA Feature Pipeline

# Check if PCA columns already exist and drop them so we don't get duplicates
cols_to_drop = [c for c in sampled_df.columns if c.startswith('pca_') or c == 'cluster']
sampled_df = sampled_df.drop(columns=cols_to_drop, errors='ignore')

from sklearn.decomposition import PCA

# Run PCA
OPTIMAL_N_COMPONENT = 50
# See above plot to find the optimal n_components
pca = PCA(n_components=OPTIMAL_N_COMPONENT)
X_pca = pca.fit_transform(X_tfidf.toarray())

# Create a df from PCA results and attach it to sampled_df
pca_cols = [f"pca_{i}" for i in range(50)]
pca_df = pd.DataFrame(X_pca, columns=pca_cols)
sampled_df = pd.concat([sampled_df, pca_df], axis=1)

# Author and Venue Embeddings
# Venue embeddings
venue_emb = sampled_df.groupby("venue")[pca_cols].mean()

# Author embeddings
authors_exploded = []
authors_emb = []
authors_exploded = sampled_df.explode('authors')
author_emb = authors_exploded.groupby('authors')[pca_cols].mean()

#### Generate Interpretable Features
These are the metadata features for trend analysis and network analysis

In [ ]:
# Number of Authors (Collaboration Size)
sampled_df['num_authors'] = sampled_df['authors'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Title Length (Character Count)
sampled_df['title_length'] = sampled_df['title'].apply(lambda x: len(str(x)))

# Number of References (How many papers this paper cites)
sampled_df['num_refs'] = sampled_df['references'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Citation Velocity (Citations per year)
sampled_df['citation_velocity'] = sampled_df['n_citation'] / (2018 - sampled_df['year'].astype(int) + 1)
print(sampled_df['citation_velocity'].max())

# Change the types as well to reduce memory
as_type = {
    "num_authors": "category",
    "title_length": np.uint16,
    "num_refs": np.uint8,
    "citation_velocity": np.float32,
}

sampled_df[["title", 'year', 'num_authors', 'num_refs', 'citation_velocity']].head()

### EDA Validation for our cleaned sampled dataset

To see if our preprocessing pipeline generated quality data

In [ ]:
# The "Power Law" Check (Citation Distribution)
# See https://link.springer.com/article/10.1007/s100510050359
# This is now done above in the preprocessing check
# Preprocessed above to remove n_citation = 50 outlier
# ARTIFACT_VAL = 50
# clean_df = sampled_df[sampled_df['n_citation'] != ARTIFACT_VAL]

plt.figure(figsize=(10,5))
sns.histplot(sampled_df['n_citation'] + 1, log_scale=True, kde=True, color='teal')
plt.title('Distribution of Citations (Log Scale)')
plt.xlabel('Number of Citations (Log)')
plt.ylabel('Count of Papers')
plt.savefig(f"{FIGURE_SAVE_PATH}/eda_citation_distribution.png")
plt.show()

plt.figure(figsize=(12, 6))
# Group by year and get mean authors
auth_trend = sampled_df.groupby('year')['num_authors'].mean().reset_index()
sns.lineplot(data=auth_trend, x='year', y='num_authors', marker='o')
plt.title('Average Number of Authors per Paper (Collaboration Trend)')
plt.ylabel('Avg Authors')
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig(f"{FIGURE_SAVE_PATH}/eda_collaboration_trend.png")
plt.show()

plt.figure(figsize=(10, 8))
# Get top 20 venues
top_venues = sampled_df['venue'].value_counts().nlargest(20).index
# Filter data to only those venues for plotting
sns.countplot(y='venue', data=sampled_df[sampled_df['venue'].isin(top_venues)], order=top_venues, palette='viridis')
plt.title('Top 20 Most Frequent Venues')
plt.savefig(f"{FIGURE_SAVE_PATH}/eda_top_venues.png")
plt.show()



# Select only numeric metadata
meta_cols = ['year', 'n_citation', 'num_authors', 'title_length', 'num_refs', 'citation_velocity']

plt.figure(figsize=(8, 6))
sns.heatmap(sampled_df[meta_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Metadata Features')
plt.savefig(f"{FIGURE_SAVE_PATH}/eda_corr_matrix.png")
plt.show()

print(sampled_df[meta_cols].corr())

## Task 2: Topic Clustering

#### K-Means Pipeline

Run clustering and name the topics

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

OPTIMAL_K = 8 # We chose this k for interpretability

# Fit Final Model
print(f"Clustering into {OPTIMAL_K} topics...")
# Use X_pca for the clustering itself (efficient)
kmeans = KMeans(n_clusters=OPTIMAL_K, random_state=42, n_init=10)
sampled_df['cluster'] = kmeans.fit_predict(X_pca)

# Interpret Clusters (Keyword Extraction)
print("Extracting top keywords for each topic...")

# We need the original TF-IDF vocabulary to convert numbers back to words
feature_names = tfidf.get_feature_names_out()

# Group papers by cluster
for i in range(OPTIMAL_K):
    # Get the indices of papers in this cluster
    cluster_indices = sampled_df[sampled_df['cluster'] == i].index

    # Calculate the AVERAGE TF-IDF vector for this cluster
    # (We use the X_tfidf matrix from Task 1, slicing by the cluster's rows)
    cluster_center = X_tfidf[cluster_indices].mean(axis=0)

    # Convert matrix to 1D array
    cluster_center_arr = np.asarray(cluster_center).flatten()

    # Get indices of the top 10 highest weighted words
    top_indices = cluster_center_arr.argsort()[::-1][:10]

    # Map indices to words
    top_keywords = [feature_names[ind] for ind in top_indices]

    print(f"\n--- Cluster {i} ({len(cluster_indices)} papers) ---")
    print(f"Keywords: {', '.join(top_keywords)}")
    # Show a sample title to verify
    print(f"Sample Title: {sampled_df[sampled_df['cluster'] == i]['title'].iloc[0]}")

For report:

Unsupervised clustering revealed a distinct group of non-english papers (cluster 1). We utilized this insight to filter the dataset, removing 186 foreign language documents to improve downstream topic modeling


Important: ONlY RUN THIS ONCE TO REMOVE THE GERMAN PAPERS!

Now we remove cluster x and rerun the pipeline

Re run TF-IDF -> PCA -> K-means pipeline

In [ ]:
# Verify the german cluster index
sampled_df[sampled_df['cluster'] == 6]

In [ ]:
german_cluster_id = 6
print(f"Removing {len(sampled_df[sampled_df['cluster'] == german_cluster_id])} non-English papers...")

sampled_df = sampled_df[sampled_df['cluster'] != german_cluster_id].copy()

sampled_df = sampled_df.reset_index(drop=True)

Visualize the clusters with new human-readable labels you might need to change the cluster_names list see above for the output result

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Update cluster names based on the output we get above
cluster_names = {
    0: 'Numerical Computing',      # matrix, inversion, polynomial
    1: 'General Modeling & Logic', # (The large catch-all cluster)
    2: 'Robotics & Control',       # robot, controller, manipulator
    3: 'Optimization & Theory',    # linear programming, equations
    4: 'Software & Info Systems',  # language, software, user, design
    5: 'Graph Theory',             # vertices, edge, connected
    6: 'Networking & Wireless',    # wireless, routing, protocol
    7: 'Editorial/Historical'      # letters to editor, algol, acm
}

# Apply mapping
sampled_df['label'] = sampled_df['cluster'].map(cluster_names)

# 3. Plot
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='pca_0',
    y='pca_1',
    hue='label',
    data=sampled_df,
    palette='tab10',
    alpha=0.9,
    s=60             # Marker size
)
plt.title('DBLP Research Landscape: Topic Clusters', fontsize=15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Research Topic')
plt.tight_layout()
plt.savefig(f"{FIGURE_SAVE_PATH}/topic_clustering_pca.png")
plt.show()

In [ ]:
from sklearn.manifold import TSNE

print("Running t-SNE to visualize clusters (this takes a moment)...")

# We use the same X_pca data, but project it differently
tsne = TSNE(n_components=2, verbose=1, perplexity=40, random_state=42)
tsne_results = tsne.fit_transform(X_pca)

# Add t-SNE results to the dataframe
sampled_df['tsne_x'] = tsne_results[:, 0]
sampled_df['tsne_y'] = tsne_results[:, 1]

# Plotting the t-SNE result
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='tsne_x',
    y='tsne_y',
    hue='label',
    data=sampled_df,
    palette='tab10', # Stronger colors
    alpha=0.7,
    s=40,
    legend='full'
)

plt.title('t-SNE Visualization of Research Topics', fontsize=16)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(f"{FIGURE_SAVE_PATH}/topic_clustering_t-SNE.png")
plt.show()

### 5. Temporal Trend Analysis

In [ ]:
# drop missing values
df = df.dropna(subset=['title', 'year', 'authors', 'references'])

# fill missing 
df['venue'] = df['venue'].fillna('Unknown Venue')
df['abstract'] = df['abstract'].fillna("")

# make sure year is integer
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df = df[df['year'].between(1950, 2017)]

In [ ]:
# create numerical features for clustering
df["text"] = (df["title"].fillna("") + " " + df["abstract"].fillna("")).str.lower()

# vectorize text using hashing
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=50000, stop_words="english", alternate_sign=False)
X_hv = hv.fit_transform(df["text"])

# dimensionality reduction using TruncatedSVD
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=150, random_state=42)
X_svd = svd.fit_transform(X_hv)
X_svd = Normalizer(copy=False).fit_transform(X_svd)

# clustering using KMeans
K = 50
kmeans = MiniBatchKMeans(n_clusters=K, random_state=42, batch_size=1024)
labels = kmeans.fit_predict(X_svd)
df['cluster'] = labels

In [ ]:
# temporal trend analysis 
trend = df.groupby(['year', 'cluster']).size().rename('count').reset_index()

year_totals = trend.groupby('year')['count'].sum().rename('year_total')
trend = trend.merge(year_totals, on='year')
trend["share"] = trend["count"] / trend["year_total"]

# visualization of trends
sns.set(style="whitegrid", context="talk", palette="muted")
top_clusters = df["cluster"].value_counts().head(10).index
trend_top = trend[trend["cluster"].isin(top_clusters)]

plt.figure(figsize=(14, 8))
for cluster in top_clusters:
    sub = trend_top[trend_top["cluster"] == cluster].sort_values("year")
    plt.plot(sub["year"], sub["share"], label=f"Cluster {cluster}", linewidth=2)

plt.axvline(2010, color="k", linestyle="--", alpha=0.5)
plt.title("Temporal Trends of Top 10 Clusters in Publications")
plt.xlabel("Year")
plt.ylabel("Share of Publications")
plt.legend(loc="upper left", ncol=2, fontsize=10, frameon=True)
plt.show()

In [ ]:
# annotate fastest rising clusters post-2010
from sklearn.linear_model import LinearRegression

def cluster_slope(cluster_id):
    sub = trend[trend["cluster"] == cluster_id]
    X = sub["year"].values.reshape(-1,1)
    y = sub["share"].values
    if len(sub) < 5:
        return np.nan
    return LinearRegression().fit(X,y).coef_[0]

slopes = {c: cluster_slope(c) for c in top_clusters}
fast_risers = sorted(slopes, key=slopes.get, reverse=True)[:3]

plt.figure(figsize=(14,8))
for c in top_clusters:
    sub = trend_top[trend_top["cluster"] == c].sort_values("year")
    plt.plot(sub["year"], sub["share"], label=f"Cluster {c}", linewidth=2)

plt.axvline(2010, color="k", linestyle="--", alpha=0.5)

for c in fast_risers:
    sub = trend_top[trend_top["cluster"] == c].sort_values("year")
    if not sub.empty:
        x, y = sub["year"].iloc[-1], sub["share"].iloc[-1]
        plt.annotate(
            f"↑ Cluster {c}",
            xy=(x,y),
            xytext=(x+1,y+0.01),
            arrowprops=dict(arrowstyle="->", color="gray")
        )

plt.title("Topic trends with post-2010 marker and fast risers")
plt.xlabel("Year")
plt.ylabel("Share of papers")
plt.legend(loc="upper left", ncol=2, fontsize=10, frameon=True)
plt.tight_layout()
plt.show()


### Task 4: Predictive Modeling (Classification)

### 7. Network Analysis